<a href="https://colab.research.google.com/github/one-pieces/google-colab/blob/main/PyTorch_%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%9F%BA%E7%A1%80%E3%80%90%E5%8A%A8%E6%89%8B%E5%AD%A6%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0_v2%E3%80%91%E6%A8%A1%E5%9E%8B%E6%94%B9%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch 神经网络基础【动手学深度学习 v2】模型改造

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0249, -0.0253,  0.2287,  0.2033, -0.2314, -0.1831,  0.0070, -0.2411,
         -0.0624,  0.1612],
        [ 0.1597, -0.0555,  0.1794,  0.0634, -0.2676, -0.3104, -0.1404, -0.1232,
         -0.2079,  0.0573]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)

  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.1369, -0.1060,  0.0999,  0.0954, -0.0040,  0.0170,  0.2150, -0.0786,
          0.1681, -0.1319],
        [ 0.1942, -0.1665, -0.0275,  0.1443, -0.1072,  0.0495,  0.2852,  0.0116,
          0.1548, -0.1941]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for block in args:
        self._modules[block] = block
    
  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3376, -0.0719, -0.2315,  0.0048,  0.2171, -0.1195,  0.0845, -0.1003,
          0.0900, -0.1064],
        [ 0.1855, -0.1155, -0.1918, -0.2293,  0.0741, -0.1637, -0.0142, -0.1371,
         -0.0759, -0.1068]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.rand_weight = torch.rand((20, 20), requires_grad = False)
    self.linear = nn.Linear(20, 20)

  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0510, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [10]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)

  def forward(self, X):
    return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1710, grad_fn=<SumBackward0>)